In [ ]:
import py3dep
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pynhd as nhd
import flopy
import geopandas as gpd
from pynhd import NLDI, NHDPlusHR, WaterData
import contextily as ctx
import numpy as np

from flopy.utils import Raster
import os

import geopandas as gpd

In [ ]:
from xrspatial import zonal_stats
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import rasterio
import xarray as xr

use xarray zonal stats to calculate minimum elelvation for grid cells


https://pygis.io/docs/e_raster_rasterize.html

In [ ]:
import basic

In [ ]:
m = basic.load_model()

In [ ]:
x = m.modelgrid.get_xcellcenters_for_layer(1)
y = m.modelgrid.get_ycellcenters_for_layer(1)


In [ ]:
mgrid = m.modelgrid

In [ ]:

dst_crs = 'EPSG:2226'
raster_ws = os.path.join("GIS", "DEM")
raster_name = "output_be_DTM.tif"
rast = os.path.join(raster_ws, raster_name)
rio = Raster.load(rast)



In [ ]:


arr = rio.get_array(1)
idx = np.isfinite(arr)

vmin, vmax = arr[idx].min(), arr[idx].max()
vmin, vmax



In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7);





In [ ]:
rio.bounds

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid)
pmv.plot_grid(ax=ax, lw=0.5, color="black");

https://pygis.io/docs/e_raster_rasterize.html

In [ ]:
grid = gpd.read_file("GIS/grid.shp")

In [ ]:

# Get list of geometries for all features in vector file
# geom = [shapes for shapes in grid.geometry]
# # create tuples of geometry, value pairs, where value is the attribute value you want to burn
# geom_value = ((geom,value) for geom, value in zip(vector.geometry, vector['id']))
geom = list((g.geometry, index) for index,g in grid.iterrows())
# Read in vector
vector = grid

# Open  raster
raster = rasterio.open(rast)

In [ ]:
# Rasterize vector using the shape and coordinate system of the raster
rasterized_grid = features.rasterize(geom,
                                out_shape = raster.shape,
                                fill = 0,
                                out = None,
                                transform = raster.transform,
                                all_touched = True,
                                default_value = 1,
                                dtype = None)

# Plot raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized_grid, ax = ax)
plt.gca().invert_yaxis()

In [ ]:
data = raster.read()[0,:,:]
data.shape

In [ ]:
rasterized_grid.shape

In [ ]:
xr_rast_grid = xr.DataArray(rasterized_grid)
xr_rast = xr.DataArray(data)

In [ ]:
zone = zonal_stats(xr_rast_grid, xr_rast)

In [ ]:
zone.head()

In [ ]:
zone_ras = zone.loc[:,'min'].values.reshape((m.dis.nrow, m.dis.ncol))

zone_ras = np.ma.array(zone_ras, mask = m.bas6.ibound.array[0]==0)

In [ ]:
zone_var = zone.loc[:,'std'].values.reshape((m.dis.nrow, m.dis.ncol))

zone_var = np.ma.array(zone_var, mask = m.bas6.ibound.array[0]==0)
# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)
ax = pmv.plot_array(
    zone_var, masked_values=rio.nodatavals, cmap = 'terrain'
)

plt.colorbar(ax, shrink=0.7);
plt.title('Model Cell Variance')

In [ ]:


# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
zone_ras = np.ma.array(zone_ras, mask = m.bas6.ibound.array[0]==0)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)
cr = pmv.plot_array(
    zone_ras, masked_values=rio.nodatavals, cmap = 'terrain',vmax = 400
)

plt.colorbar(cr, shrink=0.7);

# sfr.plot(ax = ax, zorder = 100, edgecolor = 'k')


In [ ]:
sfr_file = r"C:\modeling\MirabelWohler\SFR_files\only_sfr_lines.shp"

sfr = gpd.read_file(sfr_file)

In [ ]:
os.mkdir('RR_2022\ibound')

# setting i bound with raster elevations...

In [ ]:
cutoff = 130
mask = np.logical_or(m.bas6.ibound.array[0]==0, zone_ras>cutoff)

# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
zone_ras_filt = np.ma.array(zone_ras, mask = mask )
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)

cr = pmv.plot_array(
    np.ma.array(np.where(zone_ras>cutoff,cutoff, cutoff),mask = m.bas6.ibound.array[0]==0) , cmap = 'Reds', vmin = cutoff-1, vmax = cutoff +1)
cr = pmv.plot_array(
    zone_ras_filt, masked_values=rio.nodatavals, cmap = 'terrain',vmax = cutoff)

plt.colorbar(cr, shrink=0.7);

sfr.plot(ax = ax, zorder = 100, edgecolor = 'k')

In [ ]:
import geopandas as gpd
import os
import numpy as np
import basic
import matplotlib.pyplot as plt

In [ ]:
ibound_edit = gpd.read_file(os.path.join('GIS\ibound_update_2023.shp'))

In [ ]:
ml = basic.load_model()

In [ ]:
plt.imshow(ml.bas6.ibound.array[0])

In [ ]:
array

In [ ]:
array = np.zeros((ml.dis.nrow, ml.dis.ncol), dtype = int)
array[ibound_edit.loc[:,'row']-1, ibound_edit.loc[:,'column']-1] = ibound_edit.loc[:,'new_iboun1']
plt.imshow(array)

In [ ]:
np.savetxt('RR_2022\ibound\lay1.txt', array, fmt = "%d")

In [ ]:


# plt.figure()
# plt.imshow(~zone_ras_filt.mask)

# np.savetxt('RR_2022\ibound\lay1.txt', ~zone_ras_filt.mask, fmt = "%d")

lay2 = m.bas6.ibound.array[1]==1
lay2 = np.logical_and(~mask,lay2)
np.savetxt('RR_2022\ibound\lay2.txt', lay2, fmt = "%d")

plt.figure()
plt.imshow(lay2)
lay3 = m.bas6.ibound.array[2]==1
lay3 = np.logical_and(~mask, lay3)
np.savetxt('RR_2022\ibound\lay3.txt', lay3, fmt = "%d")
plt.figure()
plt.imshow(lay3)

In [ ]:
props = basic.load_mod_props(ml)

In [ ]:
for layer in zip((props['hk1'], props['hk2'], props['hk2']), ml.bas6.ibound.array):
    ar = np.logical_and(layer[0]==0, layer[1]==1)
    print(np.sum(np.logical_and(layer[0]==0, layer[1]==1)))
    plt.figure()
    plt.imshow(ar)
    print(ar)